In [101]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
%matplotlib inline
import matplotlib.pyplot as plt

In [102]:
# sample = pd.DataFrame({"TEC":""})
# sample

In [103]:
df = pd.read_csv("Train_TEC_mar17.csv")
df.head()

,Date,Month,Year,Time,LT,TEC
0,1,3,2017,0.016389,5.311333,2.067562
1,1,3,2017,0.049722,5.344670,2.082707
2,1,3,2017,0.083056,5.378006,2.004272
3,1,3,2017,0.116389,5.411343,1.736569
4,1,3,2017,0.149722,5.444679,1.758503


In [104]:
df.size

86400

In [105]:
df.drop(["Date","Month","Year","Time"], inplace = True, axis = 1)
df.head()

,LT,TEC
0,5.311333,2.067562
1,5.344670,2.082707
2,5.378006,2.004272
3,5.411343,1.736569
4,5.444679,1.758503


In [106]:
train = df[:-1440]
test = df[-1440:]
train.to_csv("train.csv")
test.to_csv("test.csv")

In [107]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train_tec = train.TEC.values
train_tec = train_tec.reshape((-1,1))
tec_train = train_tec
# train = train.values.reshape((-1,1))
# tec_train = train.TEC

In [108]:
test_tec = test.TEC.values
test_tec = test_tec.reshape((-1,1))
tec_test = test_tec

In [109]:
# tec_vals = df['TEC'].values
# tec_vals = tec_vals.reshape((-1,1))

# tec_train = tec_vals[:-2880]
# tec_test = tec_vals[-2880:]

lt_train = train['LT']
lt_test = test['LT']

print(len(tec_train))
print(len(tec_test))

12960
1440


In [110]:
tec_train.size

12960

In [111]:
tec_test.size

1440

In [112]:
lt_test.size, tec_test.size

(1440, 1440)

In [113]:
look_back = 0

train_generator = TimeseriesGenerator(tec_train, tec_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(tec_test, tec_test, length=look_back, batch_size=1)

In [114]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(10,
        activation='tanh',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 25
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

ValueError: slice index 0 of dimension 0 out of bounds. for 'lstm_14/strided_slice_13' (op: 'StridedSlice') with input shapes: [0,?,1], [1], [1], [1] and with computed input tensors: input[1] = <0>, input[2] = <1>, input[3] = <1>.

In [ ]:
prediction = model.predict_generator(test_generator)

tec_train = tec_train.reshape((-1))
tec_test = tec_test.reshape((-1))
prediction = prediction.reshape((-1))

plt.plot(tec_test, label='Actual')
plt.plot(prediction, label='Predicted')
plt.title("LSTM's_Prediction")
plt.xlabel('Observation')
plt.ylabel('TEC')
plt.legend()
plt.show()

In [ ]:
res

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(tec_test, prediction))

rms

In [ ]:
res = pd.DataFrame(tec_test[:1425], prediction)

In [ ]:
res.head(30)